# PROYECTO BIMESTRAL: Sistema de Recuperación de Información basado en Reuters-21578¶
## Integrantes: Madelyn Fernandez, Carlos Sanchez, Kevin Valle
### Fases del Proyecto

# 1. Adquisición de Datos


Importación de modulos y liberias a utilizar

In [41]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import nltk
import re
import string
from nltk.corpus import LazyCorpusLoader, CategorizedPlaintextCorpusReader
from nltk.stem import PorterStemmer, SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

A continuación, se presenta y obtiene los datos necesarios para la realización del Proyecto.

In [42]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/reuters/reuters/cats.txt
/kaggle/input/reuters/reuters/README
/kaggle/input/reuters/reuters/stopwords
...


Carga del corpus de Reuters utilizando un cargador diferido y especificación de cómo deben ser categorizados los archivos, así como la codificación y la ubicación de los datos.



In [43]:
# Configuración del directorio del corpus de Reuters
corpus_root = '/kaggle/input/reuters/reuters'

In [44]:
# Enumerando el id y las categorías de los primeros 5 documentos
!head -n 5 '/kaggle/input/reuters/reuters/cats.txt'

test/14826 trade
test/14828 grain
test/14829 nat-gas crude
test/14832 rubber tin sugar corn rice grain trade
test/14833 palm-oil veg-oil


In [45]:
# Cargar el corpus de Reuters
reuters = CategorizedPlaintextCorpusReader(
    corpus_root,
    r'(training|test).*',
    cat_file='cats.txt',
    encoding='ISO-8859-2'
)

In [46]:
reuters.words() #acceso a todas las palabras del corpus

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', ...]

Conteo y mostrar el número de categorías en el corpus de Reuters, y luego listar todas las categorías disponibles en el corpus.

In [47]:
num_of_cat = len(reuters.categories())
print('Numero de categorias: ' + str(num_of_cat) + ' categories')
reuters.categories() #lista de todas las categorías en el corpus.

Numero de categorias: 90 categories


['acq',
 'alum',
 'barley',
 'bop',
 'carcass',
 'castor-oil',
 'cocoa',
 'coconut',
 'coconut-oil',
 'coffee',
 'copper',
 'copra-cake',
 'corn',
 'cotton',
 'cotton-oil',
 'cpi',
 'cpu',
 'crude',
 'dfl',
 'dlr',
 'dmk',
 'earn',
 'fuel',
 'gas',
 'gnp',
 'gold',
 'grain',
 'groundnut',
 'groundnut-oil',
 'heat',
 'hog',
 'housing',
 'income',
 'instal-debt',
 'interest',
 'ipi',
 'iron-steel',
 'jet',
 'jobs',
 'l-cattle',
 'lead',
 'lei',
 'lin-oil',
 'livestock',
 'lumber',
 'meal-feed',
 'money-fx',
 'money-supply',
 'naphtha',
 'nat-gas',
 'nickel',
 'nkr',
 'nzdlr',
 'oat',
 'oilseed',
 'orange',
 'palladium',
 'palm-oil',
 'palmkernel',
 'pet-chem',
 'platinum',
 'potato',
 'propane',
 'rand',
 'rape-oil',
 'rapeseed',
 'reserves',
 'retail',
 'rice',
 'rubber',
 'rye',
 'ship',
 'silver',
 'sorghum',
 'soy-meal',
 'soy-oil',
 'soybean',
 'strategic-metal',
 'sugar',
 'sun-meal',
 'sun-oil',
 'sunseed',
 'tea',
 'tin',
 'trade',
 'veg-oil',
 'wheat',
 'wpi',
 'yen',
 'zinc']

Recorremos los primeros cinco artículos en la categoría "trade" del corpus de Reuters. Para cada artículo, imprimimos el número del artículo (de 1 a 5). Ademas, imprimimos el contenido completo (texto sin procesar) del artículo correspondiente.

In [48]:
for i in range(5):
    print('Article #' + str(i+1))
    print(reuters.raw(reuters.fileids("trade")[i]))

Article #1
ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT
  Mounting trade friction between the
  U.S. And Japan has raised fears among many of Asia's exporting
  nations that the row could inflict far-reaching economic
  damage, businessmen and officials said.
      They told Reuter correspondents in Asian capitals a U.S.
  Move against Japan might boost protectionist sentiment in the
  U.S. And lead to curbs on American imports of their products.
      But some exporters said that while the conflict would hurt
  them in the long-run, in the short-term Tokyo's loss might be
  their gain.
      The U.S. Has said it will impose 300 mln dlrs of tariffs on
  imports of Japanese electronics goods on April 17, in
  retaliation for Japan's alleged failure to stick to a pact not
  to sell semiconductors on world markets at below cost.
      Unofficial Japanese estimates put the impact of the tariffs
  at 10 billion dlrs and spokesmen for major electronics firms
  said they would virtually h

# 2. Preprocesado 

Se cargan el archivo de stopwords


In [49]:
# funcion que permite la carga de stopwords
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    return stopwords

In [50]:
# Cargar stopwords
STOPWORDS = load_stopwords('/kaggle/input/reuters/reuters/stopwords')

Se procesa los documentos.


In [51]:
codelist = ['\r', '\n', '\t']  # espacios, saltos de linea a eliminar de los docs

# funcion de analisis de texto para toda la cadena del documento
def parse_doc(text):
    text = text.lower()
    text = re.sub(r'&(.)+', "", text)  # elimina referencias con & 
    text = re.sub(r'pct', 'percent', text)  # reemplaza pct con "percent"  
    text = re.sub(r"[^\w\d'\s]+", '', text)  # elimina toda puntuación excepto el apostrofo 
    text = re.sub(r'[^\x00-\x7f]',r'', text)  # elimina caracteres no ASCII    
    if text.isdigit(): text = ""  # elimina palabras que son solo digitos 
    for code in codelist:
        text = re.sub(code, ' ', text)  # elimina caracteres de escape  
    text = re.sub('\s+', ' ', text)  # reemplaza multiples espacios con uno solo      
    return text

Se utiliza un conjunto de herramientas para limpiar y preprocesar las palabras del documento, con opciones para eliminar stopwords y realizar stemming.

In [52]:
DROP_STOPWORDS = True  #eliminacion de stopwords
STEMMING = False  #reduccion de las palabras a su raiz, definido en false para mantener los docs mas legibles 

# funcion que toma una cadena de texto y realiza un procesamiento adicional a nivel de palabras
def parse_words(text): 
    # divide el texto en palabras individuales
    tokens=text.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # elimina la puntuación de cada palabra
    tokens = [re_punc.sub('', w) for w in tokens]
    # filtra las palabras que no son alfabeticas
    tokens = [word for word in tokens if word.isalpha()]
    # filtra las palabras que tienen menos de tres caracteres
    tokens = [word for word in tokens if len(word) > 2]
    # filtra las palabras que tienen más de veinte caracteres
    tokens = [word for word in tokens if len(word) < 21]
    # elimina las stopwords si DROP_STOPWORDS es True
    if DROP_STOPWORDS:
        tokens = [w for w in tokens if not w in STOPWORDS]         
    # realiza stemming si STEMMING es True.
    if STEMMING:
        ps = PorterStemmer()
        tokens = [ps.stem(word) for word in tokens]
       # reconstruye el texto a partir de las palabras procesadas
    text = ''
    for token in tokens:
        text = text + ' ' + token
    return text  # retorna el texto procesado como una cadena

se utiliza un generador bajo demanda

In [53]:
# Generador para procesar textos
def generate_processed_texts(docs):
    for doc in docs:
        text_string = parse_doc(doc)
        text_string = parse_words(text_string)
        yield text_string

Dividimos los documentos del corpus de Reuters en conjuntos de entrenamiento (train_documents) y prueba (test_documents) basándonos en los IDs de archivo que comienzan con "trainning/" y "test/".


In [54]:
# Procesamiento de textos de entrenamiento y prueba
train_documents = [reuters.raw(fileid) for fileid in reuters.fileids() if fileid.startswith('training/')]
test_documents = [reuters.raw(fileid) for fileid in reuters.fileids() if fileid.startswith('test/')]


Imprimimos información sobre los documentos de entrenamiento del corpus.

In [55]:
print(f"Hay {len(train_documents)} noticias de Reuters.")

Hay 7769 noticias de Reuters.


uso del generador

In [56]:
# Procesamiento de textos de entrenamiento y prueba usando el generador
train_texts_generator = generate_processed_texts(train_documents)
test_texts_generator = generate_processed_texts(test_documents)

# 3. Representación de Datos en Espacio Vectorial

### BoW

In [57]:
# BOW
BOW_vectorizer = CountVectorizer()
train_vectors_bow = BOW_vectorizer.fit_transform(train_texts_generator)


### TF-IDF

In [58]:
# Reiniciar el generador para TF-IDF
train_texts_generator = generate_processed_texts(train_documents)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
train_vectors_tfidf = tfidf_vectorizer.fit_transform(train_texts_generator)

# 4. Indexación

Se construye índices invertidos para dos tipos de representaciones vectoriales de documentos: Bolsa de palabras (BOW) y TF-IDF



In [59]:
# Índice invertido BOW
indice_invertido_bow = defaultdict(list)
vocabulario_bow = BOW_vectorizer.get_feature_names_out()

for i, documento in enumerate(train_documents):
    palabras_indices = train_vectors_bow[i].nonzero()[1]
    for indice_palabra in palabras_indices:
        palabra = vocabulario_bow[indice_palabra]
        indice_invertido_bow[palabra].append(i)

In [60]:
# Índice invertido TF-IDF
vocabulario_tfidf = tfidf_vectorizer.get_feature_names_out()
indice_invertido_tfidf = defaultdict(list)

for i, documento in enumerate(train_documents):
    palabras_indices = train_vectors_tfidf[i].nonzero()[1]
    for indice_palabra in palabras_indices:
        palabra = vocabulario_tfidf[indice_palabra]
        indice_invertido_tfidf[palabra].append(i)

# 5. Diseño del Motor de Búsqueda


Implementacion de la Similitud Coseno, útil para procesar consultas de búsqueda y encontrar documentos relevantes en un conjunto de datos utilizando técnicas de recuperación de información como TF-IDF o bolsa de palabras (BOW).



In [61]:
# Procesamiento de consulta y ranking de documentos
def procesar_consulta(consulta, vectorizador, indice_invertido, train_vectors):
    consulta_procesada=parse_words(parse_doc(consulta))
    # consulta_procesada=consulta
    consulta_vectorizada = vectorizador.transform([consulta_procesada])
    consulta_indices = consulta_vectorizada.nonzero()[1]
    consulta_terminos = [vectorizador.get_feature_names_out()[i] for i in consulta_indices]

    documentos_relevantes = set()
    for termino in consulta_terminos:
        if termino in indice_invertido:
            documentos_relevantes.update(indice_invertido[termino])

    documentos_relevantes = list(documentos_relevantes)
    if not documentos_relevantes:
        return []

    relevantes = train_vectors[documentos_relevantes]
    similitudes = cosine_similarity(consulta_vectorizada, relevantes).flatten()

    return [(doc, similitud) for doc, similitud in zip(documentos_relevantes, similitudes)]


Se define una funcion que permite realizar el rankeo de los documentos, se obtiene los documentos más relevantes para una consulta, ordenados por su relevancia medida por la similitud de coseno.



In [62]:

def rankear_documentos(documentos_similitudes, nombres_archivos, top_n=5):
    documentos_similitudes.sort(key=lambda x: x[1], reverse=True)
    documentos_rankeados = [(nombres_archivos[doc], similitud) for doc, similitud in documentos_similitudes[:top_n]]
    return documentos_rankeados

In [63]:
# Obtener nombres de documentos de entrenamiento y prueba
train_docs = [os.path.basename(fileid) for fileid in reuters.fileids() if fileid.startswith('training/')]
test_docs = [os.path.basename(fileid) for fileid in reuters.fileids() if fileid.startswith('test/')]


Consultas y resultados

In [64]:
# Ejemplo de consulta del usuario
consulta_usuario = "Economic growth in Asia"

documentos_similitudes_tfidf = procesar_consulta(consulta_usuario, tfidf_vectorizer, indice_invertido_tfidf, train_vectors_tfidf)
documentos_rankeados_tfidf = rankear_documentos(documentos_similitudes_tfidf, train_docs)
documentos_similitudes_BOW = procesar_consulta(consulta_usuario, BOW_vectorizer, indice_invertido_bow, train_vectors_bow)
documentos_rankeados_BOW = rankear_documentos(documentos_similitudes_BOW, train_docs)


# print the ranked documents
print("Resultados con TF-IDF:")
for nombre_archivo, similitud in documentos_rankeados_tfidf:
    print(f"Similitud: {similitud:.4f} -> Archivo: {nombre_archivo}")
print("\n")
print("Resultados con Bag of Words:")
for nombre_archivo, similitud in documentos_rankeados_BOW:
    print(f"Similitud: {similitud:.4f} -> Archivo: {nombre_archivo}")


Resultados con TF-IDF:
Similitud: 0.2707 -> Archivo: 3246
Similitud: 0.2440 -> Archivo: 237
Similitud: 0.2437 -> Archivo: 8453
Similitud: 0.2278 -> Archivo: 11546
Similitud: 0.2209 -> Archivo: 227


Resultados con Bag of Words:
Similitud: 0.4583 -> Archivo: 3246
Similitud: 0.3997 -> Archivo: 8453
Similitud: 0.3608 -> Archivo: 11546
Similitud: 0.3484 -> Archivo: 227
Similitud: 0.3450 -> Archivo: 3838


# 6. Evaluación del Sistema

obtencion de querys de evaluacion

In [65]:
# Crear diccionario para almacenar las categorías reales y sus documentos solo para el conjunto de entrenamiento
categorias_reales = {}

# Recorrer todas las categorías en el corpus de Reuters
for categoria in reuters.categories():
    # Filtrar y extraer el número de documento de entrenamiento para la categoría actual
    categorias_reales[categoria]= set(int(fileid.split('/')[1]) for fileid in reuters.fileids(categoria) if fileid.startswith('training/'))


Función para evaluar el modelo

In [66]:
def evaluar_modelo(categorias_reales, vectorizer, indice_invertido, train_vectors, train_docs):

    test_search = {}
    
    for categoria in categorias_reales:
        consulta_usuario = procesar_consulta(categoria, vectorizer, indice_invertido, train_vectors)
        if categoria not in test_search:
            test_search[categoria] = set()
        for resultado in consulta_usuario:
            test_search[categoria].add(int(train_docs[resultado[0]]))

    return test_search


obtencion de resultados para bow y tf-idf

In [67]:
# BoW
bow_test_search = evaluar_modelo(categorias_reales, BOW_vectorizer, indice_invertido_bow, train_vectors_bow, train_docs)

#TF-IDF
tfidf_test_search = evaluar_modelo(categorias_reales, tfidf_vectorizer, indice_invertido_tfidf, train_vectors_tfidf, train_docs)



funcion para el calculo de las metricas: precisión, recall y f1

In [68]:
def calcular_metricas(categorias_diccionario, test_seach):

    metrics_by_category = defaultdict(dict)
    total_precision = 0
    total_recall = 0
    total_f1 = 0

    # Calcular métricas por categoría
    for categoria in categorias_diccionario:
        if categoria in test_seach:
            documentos_recuperados = test_seach[categoria]
            documentos_reales = categorias_diccionario[categoria]

            # Verdaderos Positivos (TP): Documentos recuperados que son relevantes
            tp = len(documentos_recuperados.intersection(documentos_reales))
            
            # Falsos Positivos (FP): Documentos recuperados que no son relevantes
            fp = len(documentos_recuperados - documentos_reales)
            
            # Falsos Negativos (FN): Documentos relevantes que no fueron recuperados
            fn = len(documentos_reales - documentos_recuperados)
            
            # Precisión: TP / (TP + FP)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            
            # Recall: TP / (TP + FN)
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            
            # F1-score: 2 * (Precisión * Recall) / (Precisión + Recall)
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # Guardar métricas por categoría
            metrics_by_category[categoria]['TP'] = tp
            metrics_by_category[categoria]['FP'] = fp
            metrics_by_category[categoria]['FN'] = fn
            metrics_by_category[categoria]['Precision'] = precision
            metrics_by_category[categoria]['Recall'] = recall
            metrics_by_category[categoria]['F1-score'] = f1_score
            
            # las métricas globales
            total_precision += precision
            total_recall += recall
            total_f1 += f1_score

    # Calcular métricas globales
    precision_global = total_precision / len(categorias_diccionario)
    recall_global = total_recall / len(categorias_diccionario)
    f1_score_global = total_f1 / len(categorias_diccionario)
    
    return metrics_by_category, precision_global, recall_global, f1_score_global

Obtencion de las metricas para bow y tf-idf

In [69]:
# Llamada a la función para calcular métricas

#BOW
metrics_by_category_bow, precision_global_bow, recall_global_bow, f1_score_global_bow = calcular_metricas(categorias_reales,bow_test_search)

#TF-IDF
metrics_by_category_tfidf, precision_global_tfidf, recall_global_tfidf, f1_score_global_tfidf = calcular_metricas(categorias_reales, tfidf_test_search)


Funciones para mostrar resultados

In [70]:
def imprimir_metricas_por_categoria(metrics_by_category):
    for categoria, metrics in metrics_by_category.items():
        print(f"Categoría: {categoria}")
        print(f"   TP: {metrics['TP']}")
        print(f"   FP: {metrics['FP']}")
        print(f"   FN: {metrics['FN']}")
        print(f"   Precisión: {metrics['Precision']:.4f}")
        print(f"   Recall: {metrics['Recall']:.4f}")
        print(f"   F1-score: {metrics['F1-score']:.4f}")
        print()


In [71]:
def imprimir_metricas_globales(precision_global, recall_global, f1_score_global):
    print("Métricas globales:")
    print(f"   Precisión Global: {precision_global:.4f}")
    print(f"   Recall Global: {recall_global:.4f}")
    print(f"   F1-score Global: {f1_score_global:.4f}")
    print()

Metricas Obtenidas de BoW y TF-IDF

In [72]:
# Imprimir métricas por categoría y globales para BoW
print("Métricas BoW:")
imprimir_metricas_por_categoria(metrics_by_category_bow)
imprimir_metricas_globales(precision_global_bow, recall_global_bow, f1_score_global_bow)


Métricas BoW:
Categoría: acq
   TP: 0
   FP: 0
   FN: 1650
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: alum
   TP: 0
   FP: 0
   FN: 35
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: barley
   TP: 37
   FP: 15
   FN: 0
   Precisión: 0.7115
   Recall: 1.0000
   F1-score: 0.8315

Categoría: bop
   TP: 0
   FP: 0
   FN: 75
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: carcass
   TP: 3
   FP: 0
   FN: 47
   Precisión: 1.0000
   Recall: 0.0600
   F1-score: 0.1132

Categoría: castor-oil
   TP: 0
   FP: 0
   FN: 1
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: cocoa
   TP: 53
   FP: 6
   FN: 2
   Precisión: 0.8983
   Recall: 0.9636
   F1-score: 0.9298

Categoría: coconut
   TP: 4
   FP: 11
   FN: 0
   Precisión: 0.2667
   Recall: 1.0000
   F1-score: 0.4211

Categoría: coconut-oil
   TP: 0
   FP: 0
   FN: 4
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: coffee
   TP: 111


In [73]:
# Imprimir métricas por categoría y globales para TF-IDF
print("Métricas TF-IDF:")
imprimir_metricas_por_categoria(metrics_by_category_tfidf)
imprimir_metricas_globales(precision_global_tfidf, recall_global_tfidf, f1_score_global_tfidf)

Métricas TF-IDF:
Categoría: acq
   TP: 0
   FP: 0
   FN: 1650
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: alum
   TP: 0
   FP: 0
   FN: 35
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: barley
   TP: 37
   FP: 15
   FN: 0
   Precisión: 0.7115
   Recall: 1.0000
   F1-score: 0.8315

Categoría: bop
   TP: 0
   FP: 0
   FN: 75
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: carcass
   TP: 3
   FP: 0
   FN: 47
   Precisión: 1.0000
   Recall: 0.0600
   F1-score: 0.1132

Categoría: castor-oil
   TP: 0
   FP: 0
   FN: 1
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: cocoa
   TP: 53
   FP: 6
   FN: 2
   Precisión: 0.8983
   Recall: 0.9636
   F1-score: 0.9298

Categoría: coconut
   TP: 4
   FP: 11
   FN: 0
   Precisión: 0.2667
   Recall: 1.0000
   F1-score: 0.4211

Categoría: coconut-oil
   TP: 0
   FP: 0
   FN: 4
   Precisión: 0.0000
   Recall: 0.0000
   F1-score: 0.0000

Categoría: coffee
   TP: 1